In [ ]:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:   This notebook contains but does not use the add_ROI(...) script
# NOTE:   The ROI was added to the datasheet outside of the notebook
# NOTE:   The processed datasheet is used to train and test the linear regression predictor
# NOTE:   The predictor code is in this notebook
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:
# NOTE:

Explore the loaded data by using the following Apache® Spark DataFrame methods:

    print schema
    print top ten records
    count all records



In [ ]:
def add_ROI(data):
    print('starting to add_roi')
    TIME_VAL_OF_MONEY = 1.05
    new_row = []
    #data = data.dropna(axis=0, how='any', subset=['term', 'total_pymnt', 'loan_amnt'])

    #data['term'] = data['term'].map(lambda x: int(x.split('months')[0]))
                                    #     data.head()
    for i, row in data.iterrows():  # change this to whatever you name the datasheet
        print(row)
        if i % 10000 == 0:
            print('i added_ROI to:', i)
        term_limit = data["term"][i]  # number of months this takes
        amount_paid = float(data['total_pymnt'][i])  # amount paid by borrower
        total_val = 0
        amount_paid_per_month = float(amount_paid) / term_limit
        
        for j in range(0, term_limit):
            years_from_end = (term_limit - j) / 12.0
            time_adjusted_val = amount_paid_per_month * (
                        TIME_VAL_OF_MONEY ** years_from_end)  # amount paid increases exponentially at a rate of TIME_VAL_OF_MONEY per year
            total_val += time_adjusted_val
        initial_investment = data['loan_amnt'][i]
        total_gain = total_val / initial_investment
        ROI = total_gain ** (1 / (term_limit / 12))  # nth root of percentage gained, where n = years of loan
        new_row.append(ROI)

    data['ROI'] = new_row  # also change this to whatever you name the sheet
    return data

print(df_data_2['term'])
add_ROI(df_data_3)

Handling missing values

df_data.show()

SMOTE to balance the dataset
Not focusing here for now. WIll come back to it later


## Create Apache Spark Machine learning model

In [10]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'QI0dGMLtvoha7CJ4aYeS3l6YOjWraYA8MMLcWBkQ0Jf2',
    'service_id': 'iam-ServiceId-a5dd711d-95a6-40de-972e-2d23e39d5d0c',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_d3956d7b54db4946b90a0cd557c288c3_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_4 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
    .option('inferSchema','True')\
  .load(cos.url('25000for_watson.csv', 'lendingclubpractice-donotdelete-pr-qmjuqkyixpbl38'))
df_data_4.take(5)


[Row(_c0=0, int_rate=10.65, annual_inc=24000.0, loan_status=1.0, dti=4.8, delinq_2yrs=0.0, label=1.0814847192525776),
 Row(_c0=1, int_rate=15.96, annual_inc=12252.0, loan_status=1.0, dti=5.105, delinq_2yrs=0.0, label=1.1053443305355617),
 Row(_c0=2, int_rate=13.49, annual_inc=49200.0, loan_status=1.0, dti=4.92, delinq_2yrs=0.0, label=1.0967721821522236),
 Row(_c0=3, int_rate=7.9, annual_inc=36000.0, loan_status=1.0, dti=7.2, delinq_2yrs=0.0, label=1.0671683506153882),
 Row(_c0=4, int_rate=18.64, annual_inc=48000.0, loan_status=1.0, dti=16.0, delinq_2yrs=0.0, label=1.123073490362814)]

In [119]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

df_no_null = df_data_4
print(df_no_null.dtypes)
df_no_null = df_no_null.drop('_c0')
df_no_null = df_no_null.drop('loan_status')
split_data = df_no_null.randomSplit([0.8, 0.20], 24)
train_data = split_data[0]
test_data = split_data[1]


print(df_no_null)

print ("Number of training records: " + str(train_data.count()))
print ("Number of testing records : " + str(test_data.count()))

[('_c0', 'int'), ('int_rate', 'double'), ('annual_inc', 'double'), ('loan_status', 'double'), ('dti', 'double'), ('delinq_2yrs', 'double'), ('label', 'double')]
DataFrame[int_rate: double, annual_inc: double, dti: double, delinq_2yrs: double, label: double]
Number of training records: 12933
Number of testing records : 3255


In [120]:
#stringIndexer_label = StringIndexer(inputCol="ROI", outputCol="label").fit(df_no_null)
#stringIndexer_loan_amnt = StringIndexer(inputCol="loan_amnt", outputCol="loan_amnt_ix")
#stringIndexer_term = StringIndexer(inputCol="term", outputCol="term_ix")
#stringIndexer_int_rate = StringIndexer(inputCol="int_rate", outputCol="int_rate_ix")
#stringIndexer_installment = StringIndexer(inputCol="installment", outputCol="installment_ix")
#stringIndexer_emp_length = StringIndexer(inputCol="emp_length", outputCol="emp_length_ix")
#stringIndexer_annual_inc = StringIndexer(inputCol="log_annual_inc", outputCol="log_annual_inc_ix")
#stringIndexer_dti = StringIndexer(inputCol="dti", outputCol="dti_ix")
#stringIndexer_fico = StringIndexer(inputCol="fico_score", outputCol="fico_score_ix")
#stringIndexer_bankrupt = StringIndexer(inputCol="pub_rec_bankruptcies", outputCol="pub_rec_bankruptcies_ix")



In [121]:
for_watson = ['int_rate', 'annual_inc', 'dti', 'delinq_2yrs']

In [122]:
vectorAssembler_features = VectorAssembler(inputCols=for_watson, outputCol="features")

In [123]:

transform_df_pipeline = Pipeline(stages=[
                              vectorAssembler_features])
transformed_df = transform_df_pipeline.fit(df_no_null).transform(df_no_null)
transformed_df = transformed_df.drop('_c0')
transformed_df = transformed_df.drop('loan_status')
transformed_df.show()

+--------+----------+------------------+-----------+------------------+--------------------+
|int_rate|annual_inc|               dti|delinq_2yrs|             label|            features|
+--------+----------+------------------+-----------+------------------+--------------------+
|   10.65|   24000.0|               4.8|        0.0|1.0814847192525776|[10.65,24000.0,4....|
|   15.96|   12252.0|             5.105|        0.0|1.1053443305355617|[15.96,12252.0,5....|
|   13.49|   49200.0|              4.92|        0.0|1.0967721821522236|[13.49,49200.0,4....|
|     7.9|   36000.0|               7.2|        0.0|1.0671683506153882|[7.9,36000.0,7.2,...|
|   18.64|   48000.0|              16.0|        0.0| 1.123073490362814|[18.64,48000.0,16...|
|   12.69|   75000.0|              6.25|        0.0|1.0783106029851526|[12.69,75000.0,6....|
|   13.49|   30000.0|3.3333333333333335|        0.0|0.6481225540561357|[13.49,30000.0,3....|
|    9.91|   15000.0|               5.0|        0.0|  1.07761019145083

In [124]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=1000, regParam=0.001, elasticNetParam=0.25)

# Fit the model
transformed_df.show()
lrModel = lr.fit(transformed_df)


+--------+----------+------------------+-----------+------------------+--------------------+
|int_rate|annual_inc|               dti|delinq_2yrs|             label|            features|
+--------+----------+------------------+-----------+------------------+--------------------+
|   10.65|   24000.0|               4.8|        0.0|1.0814847192525776|[10.65,24000.0,4....|
|   15.96|   12252.0|             5.105|        0.0|1.1053443305355617|[15.96,12252.0,5....|
|   13.49|   49200.0|              4.92|        0.0|1.0967721821522236|[13.49,49200.0,4....|
|     7.9|   36000.0|               7.2|        0.0|1.0671683506153882|[7.9,36000.0,7.2,...|
|   18.64|   48000.0|              16.0|        0.0| 1.123073490362814|[18.64,48000.0,16...|
|   12.69|   75000.0|              6.25|        0.0|1.0783106029851526|[12.69,75000.0,6....|
|   13.49|   30000.0|3.3333333333333335|        0.0|0.6481225540561357|[13.49,30000.0,3....|
|    9.91|   15000.0|               5.0|        0.0|  1.07761019145083

In [125]:
pipeline_rf = Pipeline(stages=[ vectorAssembler_features, lrModel])

In [126]:
train_data.show(10)
model_rf = pipeline_rf.fit(train_data)


+--------+----------+------------------+-----------+------------------+
|int_rate|annual_inc|               dti|delinq_2yrs|             label|
+--------+----------+------------------+-----------+------------------+
|    5.42|   14400.0| 2.057142857142857|        0.0| 1.052225767906996|
|    5.42|   15600.0|              3.12|        0.0|1.0532674891495548|
|    5.42|   16800.0|               5.6|        0.0|1.0575217123036242|
|    5.42|   16908.0|             5.636|        0.0|1.0541286715021332|
|    5.42|   17760.0| 4.059428571428572|        0.0|1.0344756971899214|
|    5.42|   18000.0|               6.0|        0.0|0.9667946153519082|
|    5.42|   18000.0|               9.0|        0.0| 1.054168406822798|
|    5.42|   18700.0|12.466666666666667|        0.0|1.0541098981667771|
|    5.42|   19008.0|             1.584|        0.0|1.0541907940614834|
|    5.42|   20000.0|2.8469750889679717|        0.0|1.0541312966520637|
+--------+----------+------------------+-----------+------------

In [127]:
print(train_data.take(1))

[Row(int_rate=5.42, annual_inc=14400.0, dti=2.057142857142857, delinq_2yrs=0.0, label=1.052225767906996)]


In [128]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model_rf.transform(test_data)
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print(predictions.take(5))
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions))


[Row(int_rate=5.42, annual_inc=12600.0, dti=6.3, delinq_2yrs=0.0, label=0.5796757598713377, features=DenseVector([5.42, 12600.0, 6.3, 0.0]), prediction=1.0289396094968732), Row(int_rate=5.42, annual_inc=16000.0, dti=2.1993127147766325, delinq_2yrs=0.0, label=1.054149268019098, features=DenseVector([5.42, 16000.0, 2.1993, 0.0]), prediction=1.0299643137671102), Row(int_rate=5.42, annual_inc=20160.0, dti=3.8037735849056604, delinq_2yrs=0.0, label=1.054180040773623, features=DenseVector([5.42, 20160.0, 3.8038, 0.0]), prediction=1.0303144211132678), Row(int_rate=5.42, annual_inc=21000.0, dti=5.0, delinq_2yrs=0.0, label=1.0327813837165414, features=DenseVector([5.42, 21000.0, 5.0, 0.0]), prediction=1.0302660833483124), Row(int_rate=5.42, annual_inc=22296.0, dti=4.128888888888889, delinq_2yrs=0.0, label=1.0513140588222238, features=DenseVector([5.42, 22296.0, 4.1289, 0.0]), prediction=1.0305622453522312)]
R Squared (R2) on test data = 0.00722128


In [129]:
#featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedterm", maxCategories=10).fit(df_no_null)

In [130]:
transform_df_pipeline = Pipeline(stages=[
                              vectorAssembler_features])
transformed_df = transform_df_pipeline.fit(df_no_null).transform(df_no_null)
transformed_df = transformed_df.drop('_c0')

transformed_df.show()



+--------+----------+------------------+-----------+------------------+--------------------+
|int_rate|annual_inc|               dti|delinq_2yrs|             label|            features|
+--------+----------+------------------+-----------+------------------+--------------------+
|   10.65|   24000.0|               4.8|        0.0|1.0814847192525776|[10.65,24000.0,4....|
|   15.96|   12252.0|             5.105|        0.0|1.1053443305355617|[15.96,12252.0,5....|
|   13.49|   49200.0|              4.92|        0.0|1.0967721821522236|[13.49,49200.0,4....|
|     7.9|   36000.0|               7.2|        0.0|1.0671683506153882|[7.9,36000.0,7.2,...|
|   18.64|   48000.0|              16.0|        0.0| 1.123073490362814|[18.64,48000.0,16...|
|   12.69|   75000.0|              6.25|        0.0|1.0783106029851526|[12.69,75000.0,6....|
|   13.49|   30000.0|3.3333333333333335|        0.0|0.6481225540561357|[13.49,30000.0,3....|
|    9.91|   15000.0|               5.0|        0.0|  1.07761019145083

In [131]:
pipeline_rf


Pipeline_4a088ee13192da13420d

In [132]:
#Ignore this cell

# from pyspark.mllib.tree import RandomForest
# model = RandomForest.trainClassifier(train_data, numClasses=2, categoricalFeaturesInfo={}, \
#     numTrees=3, featureSubsetStrategy="auto", impurity="gini", \
#     maxDepth=4, maxBins=32, seed=13579)

In [133]:
model_rf = pipeline_rf.fit(train_data)


In [134]:
predictions = model_rf.transform(test_data)
predictions = predictions.drop('_c0')

print(predictions)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print(predictions.show())



DataFrame[int_rate: double, annual_inc: double, dti: double, delinq_2yrs: double, label: double, features: vector, prediction: double]
+--------+----------+------------------+-----------+------------------+--------------------+------------------+
|int_rate|annual_inc|               dti|delinq_2yrs|             label|            features|        prediction|
+--------+----------+------------------+-----------+------------------+--------------------+------------------+
|    5.42|   12600.0|               6.3|        0.0|0.5796757598713377|[5.42,12600.0,6.3...|1.0289396094968732|
|    5.42|   16000.0|2.1993127147766325|        0.0| 1.054149268019098|[5.42,16000.0,2.1...|1.0299643137671102|
|    5.42|   20160.0|3.8037735849056604|        0.0| 1.054180040773623|[5.42,20160.0,3.8...|1.0303144211132678|
|    5.42|   21000.0|               5.0|        0.0|1.0327813837165414|[5.42,21000.0,5.0...|1.0302660833483124|
|    5.42|   22296.0| 4.128888888888889|        0.0|1.0513140588222238|[5.42,2229

### Persist created model with watson-machine-learning-client

In [135]:
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages
Requirement already up-to-date: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tabulate in /usr/local/src/conda3_runtime.v38/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: pandas in /gpfs/global_fs01/sym_s

In [136]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [137]:
wml_credentials={
  "username": "191a4d7e-10c4-456c-a6e7-8b35cf7fe71d",
  "password": "1de592a3-9d65-4458-8876-c5b310df37a7",
  "instance_id": "c549e573-c841-4eea-85be-764ef356f92c",
  "url": "https://ibm-watson-ml.mybluemix.net"
}

In [138]:
client = WatsonMachineLearningAPIClient(wml_credentials)


In [139]:
import json
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2018-06-05T15:44:20.699Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c",
    "modified_at": "2018-07-31T15:37:21.908Z",
    "guid": "c549e573-c841-4eea-85be-764ef356f92c"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "500507492",
      "type": "PAYG",
      "name": ""
    },
    "deployments": {
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "3f6acf43-ede8-413a-ac69-f8af3bb0cbfe",
    "published_models": {
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models"
    },
    "organization_guid": "f71f2ab7-9571-469a-8af3-3d618275e667",
    "plan": "lite",
    "space_guid": "ed811b8a-f0ae-4c4b-a047-718a49cedf78",
    "usage": {
      "capacity_units": {
        "current"

In [140]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "ROI"}
published_model = client.repository.store_model(model=model_rf, pipeline=pipeline_rf, meta_props=model_props, training_data=train_data)



In [141]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2018-07-31T15:38:03.604Z",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models/cd475fd9-e511-4510-8bc8-c65771774ba6",
    "modified_at": "2018-07-31T15:38:03.659Z",
    "guid": "cd475fd9-e511-4510-8bc8-c65771774ba6"
  },
  "entity": {
    "latest_version": {
      "created_at": "2018-07-31T15:38:03.659Z",
      "url": "https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/cd475fd9-e511-4510-8bc8-c65771774ba6/versions/b3995661-de79-413c-8b75-62c3a0262674",
      "guid": "b3995661-de79-413c-8b75-62c3a0262674"
    },
    "runtime_environment": "spark-2.1",
    "name": "ROI",
    "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models/cd475fd9-e511-4510-8bc8-c65771774ba6/learning_configuration",
    "tags": [],
    "model_type": "mllib-2.1",
    "input_data_schema": {
      "type": "struct",
      "fields": [
 

In [142]:
client.repository.list_models()

------------------------------------  -------------------------------------------------------------  ------------------------  ---------
GUID                                  NAME                                                           CREATED                   FRAMEWORK
cd475fd9-e511-4510-8bc8-c65771774ba6  ROI                                                            2018-07-31T15:38:03.604Z  mllib-2.1
fdec002d-892d-4a24-8667-1ba7255149ba  Lending Assist Risk prediction Random forest model 9 features  2018-07-25T19:22:35.090Z  mllib-2.1
30d0ccf7-84be-4365-bbe3-7555908fb917  Heart Failure Prediction Model                                 2018-07-23T14:30:07.786Z  mllib-2.1
695bd85a-3134-4fd8-a7db-a301bc76ee58  loan_status                                                    2018-06-28T14:30:14.765Z  wml-1.1
039cdc69-dbd3-4eef-93b3-47fb81d2aebb  titanic_survival                                               2018-06-11T19:19:45.115Z  wml-1.1
a8b2f7bc-ae4d-41ae-bd44-df8345aefced  Titanic

In [143]:
loaded_model = client.repository.load(published_model_uid)

In [144]:
test_predictions = loaded_model.transform(test_data)

In [118]:
created_deployment = client.deployments.create(published_model_uid, name="ROI prediction")


{"trace":"3504868927","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}


--------------------------
Deployment creation failed
--------------------------




WMLClientError: Deployment creation failed. Error: 402. {"trace":"3504868927","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

In [ ]:
client.deployments.list()

In [ ]:
payload_scoring = {"fields": ["loan_amnt", "term", "int_rate", "installment", "emp_length", "dti", "pub_rec_bankruptcies", "log_annual_inc", "fico_score"], "values": []}
